In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:

from langchain_openai import OpenAI, OpenAIEmbeddings
from langchain_deeplake import DeeplakeVectorStore as DeepLake
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA

In [4]:
# Before executing the following code, make sure to have your
# Activeloop key saved in the “ACTIVELOOP_TOKEN” environment variable.

# Instantiate the LLM and embeddings models
llm = OpenAI(model="gpt-3.5-turbo-instruct", temperature=0)
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

# Create our documents
texts = [
    "Napoleon Bonaparte was born in 15 August 1769",
    "Louis XIV was born in 5 September 1638"
]
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.create_documents(texts)

# Create Deep Lake dataset
my_activeloop_org_id = "sbeurnier10"
my_activeloop_dataset_name = "langchain_course_from_zero_to_hero"
dataset_path = f"hub://{my_activeloop_org_id}/{my_activeloop_dataset_name}"
db = DeepLake(dataset_path=dataset_path, embedding_function=embeddings)

# Add documents to our Deep Lake dataset
db.add_documents(docs)


In [ ]:
retrieval_qa = RetrievalQA.from_chain_type(
	llm=llm,
	chain_type="stuff",
	retriever=db.as_retriever()
)

from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType

tools = [
    Tool(
        name="Retrieval QA System",
        func=retrieval_qa.run,
        description="Useful for answering questions."
    ),
]
# Has a prompt that tells the LLM the various tools it can use and how to use them with proper input formatting and whatnot. Agent chooses tool and uses observation to either choose another tool or return final answer.
# Zero-shot means that the agent is not trained on any specific task, it just uses the tools available to it.
# React means that the agent can react to the output of the tools and use them to answer the question.
# Description means that the agent has a description of what each tool does and how to use it.
agent = initialize_agent(
	tools,
	llm,
	agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
	verbose=True
)

response = agent.run("When was Napoleone born?")
print(response)



> Entering new AgentExecutor chain...
 I should use the Retrieval QA System to find the answer.
Action: Retrieval QA System
Action Input: "When was Napoleone born?"
Observation: 
Napoleon Bonaparte was born in 15 August 1769.
Thought: I now know the final answer.
Final Answer: Napoleon Bonaparte was born in 15 August 1769.

> Finished chain.
Napoleon Bonaparte was born in 15 August 1769.


In [7]:
# load the existing Deep Lake dataset and specify the embedding function
db = DeepLake(dataset_path=dataset_path, embedding_function=embeddings)

# create new documents
texts = [
    "Lady Gaga was born in 28 March 1986",
    "Michael Jeffrey Jordan was born in 17 February 1963"
]
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.create_documents(texts)

# add documents to our Deep Lake dataset
db.add_documents(docs)

['cbb1f397-43ed-4dd1-a0ff-8fd07e4651a3',
 'e9699f03-6ba0-492c-9337-99dee1997550']

In [10]:
llm = OpenAI(model="gpt-3.5-turbo-instruct", temperature=0)

# create a retriever from the db
retrieval_qa = RetrievalQA.from_chain_type(
	llm=llm, chain_type="stuff", retriever=db.as_retriever()
)

# instantiate a tool that uses the retriever
tools = [
    Tool(
        name="Retrieval QA System",
        func=retrieval_qa.run,
        description="Useful for answering questions."
    ),
]

# create an agent that uses the tool
agent = initialize_agent(
	tools,
	llm,
	agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
	verbose=True
)
response = agent.run("When was Michael Jordan born?")
print(response)



> Entering new AgentExecutor chain...
 I should use a retrieval QA system to find the answer.
Action: Retrieval QA System
Action Input: "When was Michael Jordan born?"
Observation:  Michael Jordan was born on 17 February 1963.
Thought: I now know the final answer.
Final Answer: Michael Jordan was born on 17 February 1963.

> Finished chain.
Michael Jordan was born on 17 February 1963.


In [3]:

from langchain_openai import OpenAI

from langchain.agents import AgentType
from langchain.agents import load_tools
from langchain.agents import initialize_agent

from langchain.agents import Tool
from langchain.utilities import GoogleSearchAPIWrapper

In [6]:
search = GoogleSearchAPIWrapper()
llm = OpenAI(model="gpt-3.5-turbo-instruct", temperature=0)

In [7]:
tools = [
    Tool(
        name = "google-search",
        func=search.run,
        description="useful for when you need to search google to answer questions about current events"
    )
]

agent = initialize_agent(tools, 
                         llm, 
                         agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, 
                         verbose=True,
                         max_iterations=6)

response = agent("What's the latest news about the Mars rover?")
print(response['output'])

/var/folders/0h/1c3lg1350lq00b657b2_2rnm0000gn/T/ipykernel_2126/1136267672.py:9: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(tools,
/var/folders/0h/1c3lg1350lq00b657b2_2rnm0000gn/T/ipykernel_2126/1136267672.py:15: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = ag



> Entering new AgentExecutor chain...
 I should search for the latest news about the Mars rover.
Action: google-search
Action Input: "Mars rover news"
Observation: NASA's Curiosity rover, currently exploring Gale crater on Mars, is providing new details about how the ancient Martian climate went from potentially suitable ... Jun 23, 2025 ... NASA's Curiosity Mars rover viewed this low ridge, which looks a bit like a crumbling curb, on May 16. Scientists think the hardened edges of ... Feb 1, 2025 ... New capabilities allow the rover to do science with less energy from its batteries. Thirteen years since Curiosity landed on Mars, ... Mar 24, 2025 ... Scientists analyzing pulverized rock onboard NASA's Curiosity rover have found the largest organic compounds on the Red Planet to date. Apr 18, 2025 ... Funding: NASA, Canadian Space Agency, Science and Technology Funding Council of the United Kingdom. —Adapted from a news article first published ... Jun 2, 2025 ... The Mars Rover Design 